In [6]:
import pandas as pd

# Load data
user_skills_df = pd.read_csv(r'C:/Users/AnanyaSarkar/Documents/project/datascienceandengg/staging/raw/user_skills.csv')
completed_courses_df = pd.read_csv(r'C:/Users/AnanyaSarkar/Documents/project/datascienceandengg/staging/transformed/transformed_progress.csv')
courses_df = pd.read_csv(r'C:/Users/AnanyaSarkar/Documents/project/datascienceandengg/staging/transformed/transformed_courses.csv')

# Assume you have a DataFrame for designations and required skills
designation_skills_df = pd.DataFrame({
    'designation': ['SDE', 'SDE Sr', 'Solutions Enabler', 'Consultant', 'Solutions Architect', 'Principal Architect', 'IT', 'Finance', 'Accounting', 'HR'],
    'required_skills': [
        'Java, SQL', 
        'Java, Python', 
        'Django, Java', 
        'Agile Methodologies, Data Analysis', 
        'Cloud Computing, Solutions', 
        'Cybersecurity, IT Strategy', 
        'Data Analysis, SQL', 
        'Finance, Accounting', 
        'Accounting Principles', 
        'Employee Relations, Performance Management'
    ]
})

# Merge user skills and completed courses
user_skills_grouped = user_skills_df.groupby('userId')['skill'].apply(lambda x: ', '.join(x)).reset_index()
completed_courses_grouped = completed_courses_df.groupby('userId')['courseId'].apply(lambda x: ', '.join(x)).reset_index()

# Merge with designations and required skills
mart_user_skill_df = user_skills_grouped.merge(completed_courses_grouped, on='userId', how='left') \
    .merge(designation_skills_df, on='designation', how='left')

# Add proficiency levels and last completed course details
last_completed_courses = completed_courses_df.loc[completed_courses_df['percentage_completed'] == 100].copy()
last_completed_courses['completion_date'] = pd.to_datetime(last_completed_courses[['updated_year', 'updated_month', 'updated_day']].astype(str).agg('-'.join, axis=1))
last_completed_courses = last_completed_courses.groupby('userId').agg(
    last_completed_course=('courseId', 'first'),
    last_course_completion_date=('completion_date', 'first')
).reset_index()

mart_user_skill_df = mart_user_skill_df.merge(last_completed_courses, on='userId', how='left')

# Example skill gap calculation (placeholder logic)
mart_user_skill_df['skill_gap'] = mart_user_skill_df.apply(lambda x: ', '.join(set(x['required_skills'].split(', ')) - set(x['user_skills'].split(', '))), axis=1)

# Add a placeholder for recommended courses (this should be replaced with actual recommendation logic)
mart_user_skill_df['recommended_courses'] = "Sample Course Recommendation Logic Here"

# Save the Data Mart
mart_user_skill_df.to_csv('reporting/mart_user_skill.csv', index=False)


KeyError: 'Column not found: skill'